# Chain (w/ langgraph)


In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint

messages = [
    HumanMessage(content='배고파', name='user1'),
    AIMessage(content='뭘 먹고 싶으세요?', name='LLM'),
    HumanMessage(content='몰라', name='user1'),
    # AIMessage(content='그러면 중국집은 어떠세요?', name='LLM'),
]

for m in messages:
    m.pretty_print()

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano')
result = llm.invoke(messages)

print(type(result))
result.pretty_print()

In [ ]:
# Tool Calling (커스텀 툴 -> Docstring""" 으로 설명을 잘 써야함)

def multiply(a: int, b: int) -> int:
    """Multiply a and b
    If there is no number, transform text to proper integer

    Args:
        a: first int
        b: second int
    """    
    return a * b


llm_with_tools = llm.bind_tools([multiply])

In [ ]:
res = llm_with_tools.invoke([
    HumanMessage(content='일 곱하기 이가 뭐야?', name='user1')
])

In [ ]:
res.tool_calls

In [ ]:
res

## 메세지를 `State`로 사용하기

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages

# 그래프 턴이 반복될 때, messages가 알아서 추가되도록 하려면?
class MessagesState(TypedDict):
    # messages 키에는 AnyMessage 의 인스턴스들이 리스트로 들어간다.
    messages: Annotated[list[AnyMessage], add_messages]


# 참고용
ms = [
    AIMessage(content='뭘 도와드릴까요'),
    HumanMessage(content='그냥 도와줘')
]

add_messages(ms, AIMessage(content='??????'))


In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

# Node
def tool_calling_node(state: MessagesState):
    msgs = state['messages']
    ans = llm_with_tools.invoke(msgs)
    return {'messages': ans}


builder = StateGraph(MessagesState)
builder.add_node('tool_calling_node', tool_calling_node)

builder.add_edge(START, 'tool_calling_node')
builder.add_edge('tool_calling_node', END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))

In [56]:
res = graph.invoke({
    "messages": HumanMessage(content="배고파")
})

for m in res['messages']:
    m.pretty_print()

================================ Human Message =================================

배고파
================================== Ai Message ==================================

배고프시군요! 어떤 음식을 좋아하시나요? 아니면 간단하게 먹을 수 있는 추천 메뉴를 원하시나요?


In [ ]:
res